In [ ]:
# Summer 2021
!java -version

#Install Spark
#download file
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#extract the file
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#install findspark package
!pip install -q findspark

import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [1]:
import findspark
findspark.init()

In [2]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName("lecture18").setMaster("local[*]")#local[2]
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)

# Chapter 6. Working with Different Types of Data

In [3]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
from pyspark.sql.functions import current_date, current_timestamp
dateDf = spark.range(10)\
    .withColumn("today", current_date())\
    .withColumn("now", current_timestamp())
dateDf.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



In [7]:
from pyspark.sql.functions import date_add, date_sub, col
dateDf.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2021-07-17|        2021-07-27|
+------------------+------------------+
only showing top 1 row



In [16]:
from pyspark.sql.functions import datediff, months_between, to_date, lit
dateDf.withColumn("week_ago", date_sub(col("today"), 7))
dateDf.select(datediff(date_sub(col("today"), 7), col("today"))).show(1)

+-----------------------------------+
|datediff(date_sub(today, 7), today)|
+-----------------------------------+
|                                 -7|
+-----------------------------------+
only showing top 1 row



In [18]:
dateDf.createOrReplaceTempView("dateTable")
spark.sql('select * from dateTable').show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2021-07-22|2021-07-22 08:42:...|
|  1|2021-07-22|2021-07-22 08:42:...|
|  2|2021-07-22|2021-07-22 08:42:...|
|  3|2021-07-22|2021-07-22 08:42:...|
|  4|2021-07-22|2021-07-22 08:42:...|
|  5|2021-07-22|2021-07-22 08:42:...|
|  6|2021-07-22|2021-07-22 08:42:...|
|  7|2021-07-22|2021-07-22 08:42:...|
|  8|2021-07-22|2021-07-22 08:42:...|
|  9|2021-07-22|2021-07-22 08:42:...|
+---+----------+--------------------+



In [19]:
dateDf = dateDf.withColumn("week_ago", date_sub(col("today"), 7))
spark.sql('select * from dateTable').show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2021-07-22|2021-07-22 08:42:...|
|  1|2021-07-22|2021-07-22 08:42:...|
|  2|2021-07-22|2021-07-22 08:42:...|
|  3|2021-07-22|2021-07-22 08:42:...|
|  4|2021-07-22|2021-07-22 08:42:...|
|  5|2021-07-22|2021-07-22 08:42:...|
|  6|2021-07-22|2021-07-22 08:42:...|
|  7|2021-07-22|2021-07-22 08:42:...|
|  8|2021-07-22|2021-07-22 08:42:...|
|  9|2021-07-22|2021-07-22 08:42:...|
+---+----------+--------------------+



In [22]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01")).select(to_date(col("date"))).show()

+-------------+
|to_date(date)|
+-------------+
|   2017-01-01|
|   2017-01-01|
|   2017-01-01|
|   2017-01-01|
|   2017-01-01|
+-------------+



In [24]:
spark.sql("SELECT cast(to_date('2017-01-01', 'yyyy-mm-dd') as timestamp) as Nikita").show()

+-------------------+
|             Nikita|
+-------------------+
|2017-01-01 00:00:00|
+-------------------+



In [4]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

# Working with Strings
String manipulation shows up in nearly every data flow

In [ ]:
#The initcap function will capitalize every word in a given string when that word is separated from another by a space
from pyspark.sql.functions import initcap, col
df.select(initcap(col("Description"))).show(truncate=False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
|Set 7 Babushka Nesting Boxes       |
|Glass Star Frosted T-light Holder  |
|Hand Warmer Union Jack             |
|Hand Warmer Red Polka Dot          |
|Assorted Colour Bird Ornament      |
|Poppy's Playhouse Bedroom          |
|Poppy's Playhouse Kitchen          |
|Feltcraft Princess Charlotte Doll  |
|Ivory Knitted Mug Cosy             |
|Box Of 6 Assorted Colour Teaspoons |
|Box Of Vintage Jigsaw Blocks       |
|Box Of Vintage Alphabet Blocks     |
|Home Building Block Word           |
|Love Building Block Word           |
|Recipe Box With Metal Heart        |
+-----------------------------------+
only showing top 20 rows



In [ ]:
spark.sql("SELECT initcap(Description) FROM dfTable").show(truncate=False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
|Set 7 Babushka Nesting Boxes       |
|Glass Star Frosted T-light Holder  |
|Hand Warmer Union Jack             |
|Hand Warmer Red Polka Dot          |
|Assorted Colour Bird Ornament      |
|Poppy's Playhouse Bedroom          |
|Poppy's Playhouse Kitchen          |
|Feltcraft Princess Charlotte Doll  |
|Ivory Knitted Mug Cosy             |
|Box Of 6 Assorted Colour Teaspoons |
|Box Of Vintage Jigsaw Blocks       |
|Box Of Vintage Alphabet Blocks     |
|Home Building Block Word           |
|Love Building Block Word           |
|Recipe Box With Metal Heart        |
+-----------------------------------+
only showing top 20 rows



In [ ]:
# you can cast strings in uppercase and lowercase
from pyspark.sql.functions import lower, upper
df.select(col("Description"),
    lower(col("Description")),
    upper(lower(col("Description")))).show(2,truncate=False)

+----------------------------------+----------------------------------+----------------------------------+
|Description                       |lower(Description)                |upper(lower(Description))         |
+----------------------------------+----------------------------------+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|white hanging heart t-light holder|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |white metal lantern               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+----------------------------------+
only showing top 2 rows



In [ ]:
spark.sql("SELECT Description, lower(Description), \
Upper(lower(Description)) FROM dfTable").show(2,truncate=False)

+----------------------------------+----------------------------------+----------------------------------+
|Description                       |lower(Description)                |upper(lower(Description))         |
+----------------------------------+----------------------------------+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|white hanging heart t-light holder|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |white metal lantern               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+----------------------------------+
only showing top 2 rows



Another trivial task is adding or removing spaces around a string. You can do this by using lpad, ltrim, rpad and rtrim, trim
* pyspark.sql.functions.lpad(col, len, pad)
* Left-pad the string column to width len with pad.
* In order to add padding to the left side of the column we use left pad of column in pyspark, left padding is accomplished using lpad() function. In order to add padding to the right side of the column we use right pad of column in pyspark, right padding is accomplished using rpad() function. Let’s see how to
*  https://www.datasciencemadesimple.com/left-and-right-pad-of-column-in-pyspark-lpad-rpad/

In [ ]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
    ltrim(lit("    HELLO    ")).alias("ltrim"),
    rtrim(lit("    HELLO    ")).alias("rtrim"),
    trim(lit("    HELLO    ")).alias("trim"),
    lpad(lit("HELLO"), 10, "#").alias("lp"),
    rpad(lit("HELLO"), 10, "#").alias("rp")).show(1)

+---------+---------+-----+----------+----------+
|    ltrim|    rtrim| trim|        lp|        rp|
+---------+---------+-----+----------+----------+
|HELLO    |    HELLO|HELLO|#####HELLO|HELLO#####|
+---------+---------+-----+----------+----------+
only showing top 1 row



In [ ]:
spark.sql("SELECT \
  ltrim('    HELLLOOOO  '), \
  rtrim('    HELLLOOOO  '), \
  trim('    HELLLOOOO  '), \
  lpad('HELLOOOO  ', 3, ' '), \
  rpad('HELLOOOO  ', 10, ' ') \
FROM dfTable").show(1)

+----------------------+----------------------+---------------------+----------------------+-----------------------+
|ltrim(    HELLLOOOO  )|rtrim(    HELLLOOOO  )|trim(    HELLLOOOO  )|lpad(HELLOOOO  , 3,  )|rpad(HELLOOOO  , 10,  )|
+----------------------+----------------------+---------------------+----------------------+-----------------------+
|           HELLLOOOO  |             HELLLOOOO|            HELLLOOOO|                   HEL|             HELLOOOO  |
+----------------------+----------------------+---------------------+----------------------+-----------------------+
only showing top 1 row



Note that if lpad or rpad takes a number less than the length of the string, it will always remove values from the right side of the string.

## Regular Expressions
* Probably one of the most frequently performed tasks is searching for the existence of one string in another or replacing all mentions of a string with another value. This is often done with a tool called regular expressions that exists in many programming languages.
* There are two key functions in Spark that you’ll need in order to perform regular expression tasks: regexp_extract and regexp_replace. These functions extract values and replace values, respectively.
* Regular expressions in Java 
* https://www.vogella.com/tutorials/JavaRegularExpressions/article.html

In [ ]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
  regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
  col("Description")).show(2,truncate=False)

+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [ ]:
spark.sql("SELECT regexp_replace(Description, 'BLACK|WHITE|RED|GREEN|BLUE', 'COLOR') as \
  color_clean, Description FROM dfTable").show(2,truncate=False)

+----------------------------------+----------------------------------+
|color_clean                       |Description                       |
+----------------------------------+----------------------------------+
|COLOR HANGING HEART T-LIGHT HOLDER|WHITE HANGING HEART T-LIGHT HOLDER|
|COLOR METAL LANTERN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



Another task might be to replace given characters with other characters. Building this as a regular expression could be tedious, so Spark also provides the translate function to replace these values. This is done at the character level and will replace all instances of a character with the indexed character in the replacement string

In [ ]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
  .show(2,truncate=False)

+----------------------------------+----------------------------------+
|translate(Description, LEET, 1337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A1 1AN73RN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



In [ ]:
spark.sql("SELECT translate(Description, 'LEET', '1337'), Description FROM dfTable").show(2,truncate=False)

+----------------------------------+----------------------------------+
|translate(Description, LEET, 1337)|Description                       |
+----------------------------------+----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R|WHITE HANGING HEART T-LIGHT HOLDER|
|WHI73 M37A1 1AN73RN               |WHITE METAL LANTERN               |
+----------------------------------+----------------------------------+
only showing top 2 rows



* We can also perform something similar, like pulling out the first mentioned color
* pyspark.sql.functions.regexp_extract(str, pattern, idx)
* Extract a specific group matched by a Java regex, from the specified string column. If the regex did not match, or the specified group did not match, an empty string is returned.


Parameter	Type	Description
* **input**	string the string to search for strings matching the regular expression.
* **regex** string the regular expression to match
* **group**	integer an optional regular expression group number, defining which portion of the matching string will be returned
* https://docs.data.world/documentation/sql/reference/functions/regexp_extract.html

In [ ]:
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
     regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
     col("Description")).show(2,False)

+-----------+----------------------------------+
|color_clean|Description                       |
+-----------+----------------------------------+
|WHITE      |WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE      |WHITE METAL LANTERN               |
+-----------+----------------------------------+
only showing top 2 rows



In [ ]:
spark.sql("SELECT regexp_extract(Description, '(BLACK|WHITE|RED|GREEN|BLUE)', 1), \
  Description FROM dfTable").show(2)

+------------------------------------------------------------+--------------------+
|regexp_extract(Description, (BLACK|WHITE|RED|GREEN|BLUE), 1)|         Description|
+------------------------------------------------------------+--------------------+
|                                                       WHITE|WHITE HANGING HEA...|
|                                                       WHITE| WHITE METAL LANTERN|
+------------------------------------------------------------+--------------------+
only showing top 2 rows



In [ ]:
df2 = spark.createDataFrame([('salem 100-200',)], ['str'])
df2.show()
df2.select(regexp_extract('str', r'(\d+)-(\d+)', 1).alias('d')).show() #('(\d+)-(\d+)')

+-------------+
|          str|
+-------------+
|salem 100-200|
+-------------+

+---+
|  d|
+---+
|100|
+---+



In [ ]:
df2.select(regexp_extract('str', r'(\d+)-(\d+)', 2).alias('d')).show()

+---+
|  d|
+---+
|200|
+---+



In [ ]:
df2.select(regexp_extract('str', r'((\d+)-(\d+))', 1).alias('d')).show()

+-------+
|      d|
+-------+
|100-200|
+-------+



* ( start a capture group
* \d a shorthand character class, which matches all numbers; it is the same as [0-9]
* http://www.regular-expressions.info/charclass.html
* + one or more of the expression
* ) end a capture group
* / a literal forward slash
* If you remove the brackets (), the expression will still match, but you'll only capture one set

In [ ]:
df2 = spark.createDataFrame([('foo',)], ['str'])# foo10
df2.show()
df2.select(regexp_extract('str', r'(\d+)', 1).alias('d')).show()

+---+
|str|
+---+
|foo|
+---+

+---+
|  d|
+---+
|   |
+---+



In [ ]:
df2 = spark.createDataFrame([('aaaac',)], ['str'])
df2.select(regexp_extract('str', '(a+)(b)?(c)', 1).alias('d')).show()

+----+
|   d|
+----+
|aaaa|
+----+



In [ ]:
data = [('2345', 'Checked by John'),
('2398', 'Verified by Stacy'),
('2328', 'Verified by Srinivas than some random text'),        
('3983', 'Double Checked on 2/23/17 by Marsha')]

df2 = sc.parallelize(data).toDF(['ID', 'Notes'])

df2.show(truncate=False)

+----+------------------------------------------+
|ID  |Notes                                     |
+----+------------------------------------------+
|2345|Checked by John                           |
|2398|Verified by Stacy                         |
|2328|Verified by Srinivas than some random text|
|3983|Double Checked on 2/23/17 by Marsha       |
+----+------------------------------------------+



In [ ]:
result = df2.withColumn('Employee', regexp_extract(col('Notes'), '(.)(by)(\s+)(\w+)', 4))

result.show(truncate=False)

+----+------------------------------------------+--------+
|ID  |Notes                                     |Employee|
+----+------------------------------------------+--------+
|2345|Checked by John                           |John    |
|2398|Verified by Stacy                         |Stacy   |
|2328|Verified by Srinivas than some random text|Srinivas|
|3983|Double Checked on 2/23/17 by Marsha       |Marsha  |
+----+------------------------------------------+--------+



Here regex('(.)(by)(\s+)(\w+)') means

* (.) - Any character (except newline)
* (by) - Word by in the text
* (\s+) - One or many spaces
* (\w+) - Alphanumeric or underscore chars of length one; [a-z,A-Z,_] (the equivalent of \w) means a to z and A to Z - it isn't the literal hyphen in this context, if you did want a hyphen, put it as the first or last character of a character class.
* and group_number is 4 because group (\w+) is in 4th position in expression

Sometimes, rather than extracting values, we simply want to check for their existence. We can do this with the contains method on each column. This will return a Boolean declaring whether the value you specify is in the column’s string. In Python and SQL, we can use the instr function.

In [ ]:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite) \
  .where("hasSimpleColor")\
  .select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [ ]:
spark.sql("SELECT Description FROM dfTable \
WHERE instr(Description, 'BLACK') >= 1 OR instr(Description, 'WHITE') >= 1").show(3,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



This is trivial with just two values, but it becomes more complicated when there are values.
* locate returns the integer location (1 based location)
* pyspark.sql.functions.locate(substr, str, pos=1)
* Locate the position of the first occurrence of substr in a string column, after position pos
* You need to subtract 1 because string index starts from 1, not 0

In [ ]:
from pyspark.sql.functions import locate
df7 = spark.createDataFrame([('abcd',),('123',)], ['s',])
df7.show()
df7.select(locate('b', df7.s, 1).alias('s')).show()

+----+
|   s|
+----+
|abcd|
| 123|
+----+

+---+
|  s|
+---+
|  2|
|  0|
+---+



In [ ]:
tweet = "@__therealanna @heyitsrose let's have a zoom meeting tonite! #quarantinelife #girlsnight #onlinehangout"
df7 = spark.createDataFrame([(tweet,)], ['tweet',])
df7.show(truncate=False)

+-------------------------------------------------------------------------------------------------------+
|tweet                                                                                                  |
+-------------------------------------------------------------------------------------------------------+
|@__therealanna @heyitsrose let's have a zoom meeting tonite! #quarantinelife #girlsnight #onlinehangout|
+-------------------------------------------------------------------------------------------------------+



In [ ]:
from pyspark.sql.functions import regexp_extract,col

result = df7.withColumn('Mentions', regexp_extract(col('tweet'), '@([a-zA-Z0-9_]{1,50})', 1))

result.show(truncate=False)

+-------------------------------------------------------------------------------------------------------+-------------+
|tweet                                                                                                  |Mentions     |
+-------------------------------------------------------------------------------------------------------+-------------+
|@__therealanna @heyitsrose let's have a zoom meeting tonite! #quarantinelife #girlsnight #onlinehangout|__therealanna|
+-------------------------------------------------------------------------------------------------------+-------------+



In [ ]:
import  pyspark.sql.functions as F
 
df = spark.createDataFrame([('2 AVENUE DES LAPINOUS',)], ['col'])
df.show(truncate=False)
df = df.withColumn('output', F.expr("regexp_extract_all(col, '(\\\\w+)', 1)"))

df.show(truncate=False)

+---------------------+
|col                  |
+---------------------+
|2 AVENUE DES LAPINOUS|
+---------------------+

+---------------------+--------------------------+
|col                  |output                    |
+---------------------+--------------------------+
|2 AVENUE DES LAPINOUS|[2, AVENUE, DES, LAPINOUS]|
+---------------------+--------------------------+



In [ ]:
result = df7.withColumn('Mentions', F.expr("regexp_extract_all(tweet, '@([a-zA-Z0-9_]{1,50})', 1)"))
result.show(truncate=False)

+-------------------------------------------------------------------------------------------------------+---------------------------+
|tweet                                                                                                  |Mentions                   |
+-------------------------------------------------------------------------------------------------------+---------------------------+
|@__therealanna @heyitsrose let's have a zoom meeting tonite! #quarantinelife #girlsnight #onlinehangout|[__therealanna, heyitsrose]|
+-------------------------------------------------------------------------------------------------------+---------------------------+



In [ ]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
  #x=locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string)
  #df.select(x).show()
  return locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("Description")) # has to a be Column type

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
  .select("Description").show(5, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
|HAND WARMER RED POLKA DOT         |
|RED COAT RACK PARIS FASHION       |
+----------------------------------+
only showing top 5 rows



In [ ]:
x=[1,2,3,4]
x.append(7)
x

[1, 2, 3, 4, 7]

In [ ]:
print(*selectedColumns)

Column<'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`'> Column<'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`'> Column<'CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`'> Column<'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`'> Column<'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`'> Column<'unresolvedstar()'>


In [ ]:
df.select(*selectedColumns).count()

3108

In [ ]:
df.select(*selectedColumns).where(expr("is_white OR is_red")).count()

428